In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from art.attacks import FastGradientMethod,BasicIterativeMethod,DeepFool,BoundaryAttack,SpatialTransformation,NewtonFool
from art.attacks import SaliencyMapMethod,ProjectedGradientDescent,ElasticNet,CarliniLInfMethod,CarliniL2Method
from art.classifiers import PyTorchClassifier
import time
import datetime
import torchvision
import torch
import torchvision.transforms as transforms
from copy import deepcopy
import seaborn as sns
import matplotlib.pyplot as plt
from load_model_37 import load_model  # XX is the digit for python number, e.g. 37
import pandas as pd
import matplotlib

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

def de_normalize(image):
    image2 = deepcopy(image)
    for i in range(3):
        image2[i] = image2[i]*std[i]+mean[i]
    return image2

def load_data(data_dir):
    data_loader = torch.utils.data.DataLoader(
        ImageFolderWithPaths(data_dir, transforms.Compose([
            transforms.Resize(128),
            transforms.CenterCrop(128),
            transforms.ToTensor(),
            normalize,])),
            batch_size=100, 
            num_workers=0)
    for data,target,path in data_loader:
        return data,target,path

def clipmax(mean,std):
    return (1-mean)/std

def clipmin(mean,std):
    return (0-mean)/std

def clipping(shape):
    clip = np.zeros(shape)
    for i in range(3):
        clip[0,i,:,:]=clipmin(mean[i],std[i])
        clip[1,i,:,:]=clipmax(mean[i],std[i])
    return clip

def generate_adv(in_img,adv_crafter,classifier):
    out_img = adv_crafter.generate(x=in_img.numpy())
    preds = np.argmax(classifier.predict(out_img), axis=1)
    return out_img,preds

def compare_L2(in_img, out_img,preds,target,start):
    # To compare L2 norm
    count = 0
    L2 = []
    failed = []
    successful = []
    for i in range(len(out_img)):
        image1 = de_normalize(in_img.numpy()[i])
        image2 = de_normalize(out_img[i])
        if preds[i]!=target[i]:
            L2.append(np.linalg.norm(image1-image2))
            count+=1
            successful.append(i)
        else:
            failed.append(i)
    if count!=0:
        print('Fool rate: {}%,'.format(int(count*100/len(out_img))),
              'time taken: {:.1f}min,'.format((time.time() - start)/60),
             "L2 norm per image: {:.2f}".format(sum(L2)/count))
    else:
        print('Fool rate: {}%,'.format(int(count*100/len(out_img))),
              'time taken: {:.1f}min,'.format((time.time() - start)/60),
             "no image fooled.")
#     print('Failed images    :',failed)
#     print('Successful images    :',successful)
#     print('Preds: ',preds)
    return L2, failed
def generate_adv_img(method,var,LR,EPS):
    start_global = time.time()
    print(method)
    
    # Load data
    data_dir = 'data/clean_images/'
    data,target,path = load_data(data_dir)

    # Load model
    model = load_model("model/model.pt", 'cpu')

    # Create clipping values (2,3,128,128)
    shape = (2,3,128,128)
    clip = clipping(shape)

    # Start fooling
    model.eval()
    criterion = nn.CrossEntropyLoss()
    classifier = PyTorchClassifier(model=model, clip_values=clip, loss=criterion,optimizer=None, input_shape=(3, 128, 128), nb_classes=4, preprocessing=(0,1))
    classifier.set_learning_phase(False)

    in_img = deepcopy(data) # Choose images to fool
    L2_matrix = np.zeros((len(var),100)) # Store L2_values of different variables
    img_matrix = np.zeros((len(var),100,3,128,128))
    pred_matrix = np.zeros((len(var),100))

    for i in range(len(var)):
        print('Start fooling with {}, variable: {}'.format(method,var[i]))
        start = time.time()    
        # Choose classifier
        if method == 'FGSM':
            adv_crafter = FastGradientMethod(classifier, eps=var[i])
        elif method == 'Carlini LR':
            adv_crafter = CarliniLInfMethod(classifier, learning_rate=var[i], eps=EPS)
        elif method == 'Carlini EPS':
            adv_crafter = CarliniLInfMethod(classifier, learning_rate=LR, eps=var[i])
        elif method == 'DeepFool':
            adv_crafter = DeepFool(classifier, max_iter=var[i], epsilon=EPS, nb_grads=1, batch_size=1)
        else:
            print('Method not specified correctly, using Newton Fool by default')
            adv_crafter = NewtonFool(classifier)
        out_img,preds = generate_adv(in_img,adv_crafter,classifier)
        L2, failed = compare_L2(in_img, out_img,preds,target,start)

        for j in range(100):
            if j in failed:
                img_matrix[i,j] = out_img[j]
                pred_matrix[i,j] = preds[j]            
                pass
            else:
                L2_matrix[i,j] = L2.pop(0)
                img_matrix[i,j] = out_img[j]
                pred_matrix[i,j] = preds[j]
    
    print('Total time taken: {:.1f}min\n'.format((time.time()-start_global)/60))
    return L2_matrix, img_matrix, pred_matrix

def npconcat(L2, img, adv_label,L2_master, img_master, adv_label_master):
    L2_master = np.concatenate((L2_master,L2),axis=0)
    img_master = np.concatenate((img_master,img),axis=0)
    adv_label_master = np.concatenate((adv_label_master,adv_label),axis=0)
    return L2_master, img_master, adv_label_master

In [3]:
# To a library of generate_adv_img(method,var,LR,EPS), total time 2 hours
# 88 images for each original image and choose the best one

# Generate 1 FGSM sample
var = [0.001]
L2_master, img_master, adv_label_master = generate_adv_img('FGSM',var,None,None)

# Generate multiple FGSM adv images
var = [0.002,0.003,0.004,0.005,0.01,0.05,0.1,0.15,0.2,0.3,0.4,0.5,1]
L2, img, adv_label = generate_adv_img('FGSM',var,None,None)
L2_master, img_master, adv_label_master = npconcat(L2, img, adv_label,L2_master, img_master, adv_label_master)


# Generate Carlini adv images with different learning rate and eps
var = [0.005,0.01,0.05,0.1,0.15,0.2,0.3,0.4,0.5,1]
for lr in [0.001,0.005,0.01,0.05,0.1,0.3,0.5]:
    L2, img, adv_label = generate_adv_img('Carlini EPS',var,lr,None)
    L2_master, img_master, adv_label_master = npconcat(L2, img, adv_label,L2_master, img_master, adv_label_master)

# Generate Deep Fool adv images
var = [20,30,45,50]
L2, img, adv_label = generate_adv_img('DeepFool',var,None,1e-6)
L2_master, img_master, adv_label_master = npconcat(L2, img, adv_label,L2_master, img_master, adv_label_master)

torch.save(L2_master, 'L2.pt')
np.save('img',img_master) #using npy as pt couldn't store large files
torch.save(adv_label_master, 'adv_label.pt')

FGSM
Start fooling with FGSM, variable: 0.001
Fool rate: 1%, time taken: 0.4min, L2 norm per image: 0.05
Total time taken: 0.4min

FGSM
Start fooling with FGSM, variable: 0.002
Fool rate: 2%, time taken: 0.4min, L2 norm per image: 0.09
Start fooling with FGSM, variable: 0.003
Fool rate: 5%, time taken: 0.4min, L2 norm per image: 0.14
Start fooling with FGSM, variable: 0.004
Fool rate: 6%, time taken: 0.4min, L2 norm per image: 0.19
Start fooling with FGSM, variable: 0.005
Fool rate: 7%, time taken: 0.4min, L2 norm per image: 0.24
Start fooling with FGSM, variable: 0.01
Fool rate: 11%, time taken: 0.4min, L2 norm per image: 0.48
Start fooling with FGSM, variable: 0.05
Fool rate: 64%, time taken: 0.4min, L2 norm per image: 2.37
Start fooling with FGSM, variable: 0.1
Fool rate: 77%, time taken: 0.4min, L2 norm per image: 4.74
Start fooling with FGSM, variable: 0.15
Fool rate: 83%, time taken: 0.4min, L2 norm per image: 7.12
Start fooling with FGSM, variable: 0.2
Fool rate: 83%, time taken

In [3]:
# To load the data instead of running the script above which takes 2 hours
L2 = torch.load('L2.pt')
img = np.load('img.npy') # This is 3gb file size
adv_label = torch.load('adv_label.pt')

In [4]:
# Found that if both labels are classified almost equally, it tends to get wrong
# There are 3 images that have this issue, image 50,52 and 80
# For image 50, the 4th lowest L2 image is adv image
# For image 52, the 2nd lowest L2 image is adv image
# For image 80, the 5th lowest L2 image is adv image

# Get the best images
L2_clean = np.where(L2==0, 100, L2)
index = np.argmin(L2_clean, axis=0)

# Get the 2nd best images
for idx in range(100):
    L2_clean[index[idx],idx] = 99
index2 = np.argmin(L2_clean, axis=0)

# Get the 3rd best images
for idx in range(100):
    L2_clean[index2[idx],idx] = 99
index3 = np.argmin(L2_clean, axis=0)

# Get the 4rd best images
for idx in range(100):
    L2_clean[index3[idx],idx] = 99
index4 = np.argmin(L2_clean, axis=0)

# Get the 5th best images
for idx in range(100):
    L2_clean[index4[idx],idx] = 99
index5 = np.argmin(L2_clean, axis=0)

In [6]:
# To save images into pt file as normalized images

data_dir = 'data/clean_images/'
_,_,path = load_data(data_dir)
model = load_model("model/model.pt", 'cpu')

best_adv_label = []
best_L2 = []
best_img = np.zeros((100,3,128,128))

for i in range(100):
    if (model(torch.from_numpy(img[index[i],i])[None,:].float()).max(axis=1)[1]==adv_label[index[i],i]):        
        best_adv_label.append(adv_label[index[i],i])
        best_L2.append(L2[index[i],i])
        best_img[i,:,:,:] = img[index[i],i]
        img_pathname = path[i].replace('clean','adv')
        img_pathname = img_pathname.replace('.png','.pt')
        torch.save(img[index[i],i],img_pathname)
    else:
        if i==50:
            best_adv_label.append(adv_label[index4[i],i])
            best_L2.append(L2[index4[i],i])
            best_img[i,:,:,:] = img[index4[i],i]
            img_pathname = path[i].replace('clean','adv')
            img_pathname = img_pathname.replace('.png','.pt')
            torch.save(img[index4[i],i],img_pathname)
        elif i==52:
            best_adv_label.append(adv_label[index2[i],i])
            best_L2.append(L2[index2[i],i])
            best_img[i,:,:,:] = img[index2[i],i]
            img_pathname = path[i].replace('clean','adv')
            img_pathname = img_pathname.replace('.png','.pt')
            torch.save(img[index2[i],i],img_pathname)
        elif i==80:
            best_adv_label.append(adv_label[index5[i],i])
            best_L2.append(L2[index5[i],i])
            best_img[i,:,:,:] = img[index5[i],i]
            img_pathname = path[i].replace('clean','adv')
            img_pathname = img_pathname.replace('.png','.pt')
            torch.save(img[index5[i],i],img_pathname)

In [68]:
# Load clean images
data_dir = 'data/clean_images/'
clean_img,_,path = load_data(data_dir)

# Load adversarial images (pt file)
adv_img = np.zeros((100,3,128,128))
img_no = 0
for path_ in path:
    path_ = path_.replace('clean','adv')
    path_ = path_.replace('png','pt')
    adv_img[img_no,:,:,:] = torch.load(path_)
    img_no +=1

# Convert image to tensor
adv_img = torch.from_numpy(adv_img).float()

# Calculate L2
minL2 = 0
for i in range(100):
    minL2 += np.linalg.norm(de_normalize(clean_img[i])-de_normalize(adv_img[i]))

# Generate adv_img labels
print('Original label 0\nAdversarial label:',model(adv_img).max(axis=1)[1][:25])
print('Original label 1\nAdversarial label:',model(adv_img).max(axis=1)[1][25:50])
print('Original label 2\nAdversarial label:',model(adv_img).max(axis=1)[1][50:75])
print('Original label 3\nAdversarial label:',model(adv_img).max(axis=1)[1][75:])
print('min L2 :',minL2/100)

Original label 0
Adversarial label: tensor([1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1,
        1])
Original label 1
Adversarial label: tensor([3, 0, 0, 3, 0, 2, 0, 3, 2, 2, 0, 3, 2, 3, 2, 2, 0, 3, 0, 3, 2, 2, 3, 0,
        3])
Original label 2
Adversarial label: tensor([0, 1, 1, 3, 0, 3, 1, 1, 3, 3, 1, 1, 0, 3, 3, 1, 1, 1, 1, 3, 1, 0, 1, 1,
        3])
Original label 3
Adversarial label: tensor([1, 0, 1, 0, 0, 2, 2, 1, 0, 2, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 0, 1, 1, 2,
        1])
min L2 : 1.0811090415157378
